In [19]:
import psycopg2
import glob


In [20]:
dbname = 'postgres'
user = 'postgres'
password = 'recommendation_films_oct_23_MLOPS'
host = 'reco-films-db.ck2uuvj8tg5b.eu-west-3.rds.amazonaws.com'

In [21]:
# Remplacez ces valeurs par vos informations de connexion
dbname = 'postgres'
user = 'postgres'
password = 'recommendation_films_oct_23_MLOPS'
host = 'localhost'


In [22]:
sql_file = sorted(glob.glob("./sql/*"))

In [23]:
def executesql(fichier_sql):
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    # Créer un objet cursor
    cur = conn.cursor()

    # Ouvrir le fichier SQL et exécuter son contenu
    with open(fichier_sql, 'r') as file:
        # Divise le script en commandes séparées par ';'
        sql_script = file.read()
        commands = sql_script.split(';')

        # Exécute chaque commande individuellement
        for command in commands:
            if command.strip() != '':  # Vérifie si la commande n'est pas vide
                    print(command)
                    cur.execute(command)
                    conn.commit()
    # Fermer la communication avec la base de données
    conn.commit()
    cur.close()
    conn.close()

In [24]:

for file in sql_file:
        print(file)
        executesql(file)


./sql/1-create_database.sql
CREATE TABLE genre (
	genreid serial4 NOT NULL,
	"name" varchar NOT NULL,
	CONSTRAINT genre_pkey PRIMARY KEY (genreid)
)


CREATE TABLE movie (
	movieid serial4 NOT NULL,
	title varchar NOT NULL,
	imdbid int4 NULL,
	tmdbid int4 NULL,
	"year" int4 NULL,
	decade int4 NULL,
	CONSTRAINT movie_pkey PRIMARY KEY (movieid)
)

CREATE TABLE imdb_data (
	movieid int4 NOT NULL,
	titre varchar(255) NULL,
	summary text NULL,
	certificat varchar(50) NULL,
	duration varchar(10) NULL,
	poster text NULL,
	directors varchar(500) NULL,
	writers varchar(500) NULL,
	stars varchar(500) NULL,
	CONSTRAINT imdb_data_pkey PRIMARY KEY (movieid),
	CONSTRAINT imdb_data_movieid_fkey FOREIGN KEY (movieid) REFERENCES movie(movieid)
)

CREATE INDEX movie_decade_idx ON movie USING btree (decade)

CREATE INDEX movie_year_idx ON movie USING btree (year)

CREATE TABLE model_prediction (
	modelid serial4 NOT NULL,
	class_name varchar NULL,
	params varchar NULL,
	mlflow_run_id varchar NULL,
	mlflo

ProgrammingError: can't execute an empty query

In [95]:
import pandas as pd

In [96]:
rating = pd.read_csv("./ml-latest-small/ratings.csv")
movie = pd.read_csv("./ml-latest-small/movies.csv")
link= pd.read_csv("./ml-latest-small/links.csv")

In [97]:
import numpy as np

In [98]:
movie=movie.merge(link,on="movieId")
movie=movie.replace(np.nan,"")

In [99]:
movie[ movie['movieId']==791 ]

,movieId,title,genres,imdbId,tmdbId
624,791,"Last Klezmer: Leopold Kozlowski, His Life and ...",Documentary,113610,


In [100]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [101]:
(1234//10)*10

1230

In [102]:
import re

def insert_sql_movie(row):
    with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
        with conn.cursor() as cur:
            insert_query = """INSERT INTO movie (movieid, title, imdbid, tmdbid, year, decade)
            VALUES (%s, %s, %s, %s, %s, %s);
            """
            title = row["title"]
            match = re.search(r'\((\d{4})\)', title)
            year = int(match.group(1)) if match else None
            decade = (year // 10) * 10 if year else None
            
            # It's a good idea to check if year is None before proceeding
            if year is None:
                # Handle the case where the year couldn't be extracted
                print(f"Year could not be extracted for title: {title}")
                return

            data_to_insert = (
                int(row["movieId"]),
                row["title"],
                row["imdbId"],
                row["tmdbId"] if row["tmdbId"] else None,
                year,
                decade
            )
            cur.execute(insert_query, data_to_insert)

In [103]:
def insert_sql_rating(row):
    try:
        # Utilisation de gestionnaires de contexte pour la connexion et le curseur
        with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                insert_query = """INSERT INTO rating (userid, movieid, rating, timestamp)
                                  VALUES (%s, %s, %s, %s);"""
                data_to_insert = (
                    int(row["userId"]),
                    int(row["movieId"]),  # Assurez-vous que movieId est également un entier
                    row["rating"],
                    row["timestamp"]
                )
                cur.execute(insert_query, data_to_insert)
    except Exception as e:
        # Gestion des erreurs en affichant un message d'erreur
        print(f"An error occurred: {e}")

In [104]:
movie.apply(insert_sql_movie,axis=1)

Year could not be extracted for title: Babylon 5
Year could not be extracted for title: Ready Player One
Year could not be extracted for title: Hyena Road
Year could not be extracted for title: The Adventures of Sherlock Holmes and Doctor Watson
Year could not be extracted for title: Nocturnal Animals
Year could not be extracted for title: Paterson
Year could not be extracted for title: Moonlight
Year could not be extracted for title: The OA
Year could not be extracted for title: Cosmos
Year could not be extracted for title: Maria Bamford: Old Baby
Year could not be extracted for title: Death Note: Desu nôto (2006–2007)
Year could not be extracted for title: Generation Iron 2
Year could not be extracted for title: Black Mirror


0       None
1       None
2       None
3       None
4       None
        ... 
9737    None
9738    None
9739    None
9740    None
9741    None
Length: 9742, dtype: object

In [105]:
rating.apply(insert_sql_rating,axis=1)

An error occurred: insert or update on table "rating" violates foreign key constraint "rating_movieid_fkey"
DETAIL:  Key (movieid)=(176601) is not present in table "movie".

An error occurred: insert or update on table "rating" violates foreign key constraint "rating_movieid_fkey"
DETAIL:  Key (movieid)=(147250) is not present in table "movie".

An error occurred: insert or update on table "rating" violates foreign key constraint "rating_movieid_fkey"
DETAIL:  Key (movieid)=(171749) is not present in table "movie".

An error occurred: insert or update on table "rating" violates foreign key constraint "rating_movieid_fkey"
DETAIL:  Key (movieid)=(171631) is not present in table "movie".

An error occurred: insert or update on table "rating" violates foreign key constraint "rating_movieid_fkey"
DETAIL:  Key (movieid)=(171891) is not present in table "movie".

An error occurred: insert or update on table "rating" violates foreign key constraint "rating_movieid_fkey"
DETAIL:  Key (movieid)

0         None
1         None
2         None
3         None
4         None
          ... 
100831    None
100832    None
100833    None
100834    None
100835    None
Length: 100836, dtype: object

In [106]:
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                query = """select * from genre;"""
                cur.execute(query=query)
                rows = cur.fetchall()
                movie_genres = pd.DataFrame()
                for row in rows:
                    movie_genre = movie[ movie['genres'].str.contains(row[1], case=False, na=False)][["movieId"]]
                    movie_genre["genreId"]=[row[0]]*len(movie_genre)
                    movie_genres=pd.concat((movie_genres,movie_genre),axis=0)
                

/var/folders/y5/xr83lb_x16b7sc28gss89mrm0000gn/T/ipykernel_21289/982665775.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  movie_genre = movie[ movie['genres'].str.contains(row[1], case=False, na=False)][["movieId"]]


In [107]:
movie_genres

,movieId,genreId
8517,114335,1
8684,122888,1
8687,122896,1
8782,129250,1
8836,132084,1
...,...,...
9164,148652,20
9267,157108,20
9366,162350,20
9520,171759,20


In [108]:
def insert_sql_genre(row):
    try:
        with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                insert_query = """INSERT INTO movie_genre ( movieid, genreid)
                                  VALUES (%s, %s);"""
                cur.execute(insert_query,(int(row["movieId"]),int(row['genreId'])))
    except Exception:
        print(
              (
               int(row["movieId"]),
               int(row['genreId'])
               )
              )

movie_genres.apply(insert_sql_genre,axis=1)


(143410, 1)
(147250, 1)
(156605, 1)
(167570, 1)
(171495, 1)
(171631, 1)
(171749, 1)
(171891, 1)
(176601, 1)
(140956, 2)
(149334, 9)
(162414, 9)
(40697, 17)
(140956, 17)
(140956, 18)
(149334, 18)


8517    None
8684    None
8687    None
8782    None
8836    None
        ... 
9164    None
9267    None
9366    None
9520    None
9581    None
Length: 22084, dtype: object

In [109]:
#Refresh de la vue table_recap_view
try:
    with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
        with conn.cursor() as cur:
            refresh_query = """REFRESH MATERIALIZED VIEW table_recap_view;"""
            cur.execute(refresh_query)
except Exception as e:
    print("Error for refresh Table Recap {e}")